In [3]:
import torch
import glob as gb
import numpy as np
from PIL import Image
import os
import random
import torch.nn as nn
import pandas as pd 
import glob as gb

import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn

from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

#from skimage.measure import compare_psnr as psnr
#from skimage.measure import compare_ssim as ssim

os.sys.path.append('../')
from utils.loss import cos_sim

random.seed(1)
np.random.seed(1)

In [6]:
# SR Model Initialization

GPU_IN_USE = torch.cuda.is_available()

if GPU_IN_USE:
    cudnn.benchmark = True
    
device = torch.device('cuda' if GPU_IN_USE else 'cpu')

In [16]:
# Testing upsampling via SR
class SuperResolveDL:
    def __init__(self, model, size, channel, resize=True):
        self.model = model
        self.size = size
        self.channel = channel
        self.resize = resize
        
    def __call__(self, path=''):
        if self.channel == 1:
            img = Image.open(path).convert('YCbCr').resize((self.size,self.size),resample=Image.BICUBIC)
            upsample, cb, cr = img.split()
            if not self.resize:
                upsample, _, _ = Image.open(path).convert('YCbCr').split()
        elif self.channel == 3:
            if self.resize: 
                upsample = Image.open(path).convert('RGB').resize((self.size,self.size),resample=Image.BICUBIC)
            else:
                upsample = Image.open(path).convert('RGB')
        else:
            raise Exception('Channel option not listed')
        
        data = (transforms.ToTensor()(upsample)).view(1, self.channel, upsample.size[1], upsample.size[0])
        data = data.to(device)
        out = self.model(data)
        out = out.cpu()
        out_img = out.data[0].numpy()
        out_img *= 255.0
        out_img = out_img.clip(0, 255)
                
        if self.channel == 1:
            out_img = Image.fromarray(np.uint8(out_img[0]), mode='L')
            out_img = Image.merge('YCbCr', [out_img, cb, cr]).convert('RGB')
        
        if self.channel == 3:
            out_img = out_img.transpose((1,2,0))
            out_img = Image.fromarray(np.uint8(out_img), mode='RGB')

        return out_img
    
    
# Testing upsampling via Bicubic

def load_image(path='', size=224, resize=True): # Bicubic
    img = Image.open(path)
    if resize:
        img = img.resize((size,size), resample=Image.BICUBIC)
    return img

In [7]:
real_img_path = '/media/angelo/DATEN/Datasets/CelebA/HR/test/'
real_imgs = gb.glob(real_img_path + '*.jpg')

upx2_img_path = '/media/angelo/DATEN/Datasets/CelebA/LR_112/test/'
upx2_imgs = gb.glob(upx2_img_path + '*.jpg')

upx4_img_path = '/media/angelo/DATEN/Datasets/CelebA/LR_56/test/'
upx4_imgs = gb.glob(upx4_img_path + '*.jpg')

## Task 1 - Upsample x4

In [5]:
# SubCNN 
model_1_path = '../models/SubCNN_coord_x4_epoch_50.pth'
model_1 = torch.load(model_1_path, map_location=lambda storage, loc: storage.cuda(0))
model_1 = model_1.to(device)

load_SR = SuperResolveDL(model_1, size=224, channel=1, resize=False)

In [6]:
bicubic_psnr = 0
SR_psnr = 0
bicubic_ssim = 0
SR_ssim = 0

for index in range(len(real_imgs)):
    
    bicubic = np.array(load_image(upx4_imgs[index], size=224, resize=True))
    SR_img = np.array(load_SR(upx4_imgs[index]))
    real_img = np.array(load_image(real_imgs[index], resize=False))
    
    bicubic_psnr += psnr(bicubic, real_img)
    SR_psnr += psnr(SR_img, real_img)
    
    bicubic_ssim += ssim(bicubic, real_img, multichannel=True)
    SR_ssim += ssim(SR_img, real_img, multichannel=True)
    
    #print('Bicubic PSNR {0:.2f}'.format(bicubic_psnr))
    #print('SR Model PSNR {0:.2f}'.format(SR_psnr))
    #print('Bicubic SSIM {0:.2f}'.format(bicubic_ssim))
    #print('SR Model SSIM {0:.2f}'.format(SR_ssim))
    
print('Average Bicubic PSNR {0:.2f}'.format(bicubic_psnr/len(real_imgs)))
print('Average SR Model PSNR {0:.2f}'.format(SR_psnr/len(real_imgs)))
print('Average Bicubic SSIM {0:.2f}'.format(bicubic_ssim/len(real_imgs)))
print('Average SR Model SSIM {0:.2f}'.format(SR_ssim/len(real_imgs)))    

Average Bicubic PSNR 27.93
Average SR Model PSNR 28.13
Average Bicubic SSIM 0.79
Average SR Model SSIM 0.80


### SRCNN Coord

In [7]:
model_2_path = '../models/SRCNN_coord_x4_epoch_50.pth'
model_2 = torch.load(model_2_path, map_location=lambda storage, loc: storage.cuda(0))
model_2 = model_2.to(device)

load_SR = SuperResolveDL(model_2, size=224, channel=1, resize=True)

In [8]:
bicubic_psnr = 0
SR_psnr = 0
bicubic_ssim = 0
SR_ssim = 0

for index in range(len(real_imgs)):
    
    bicubic = np.array(load_image(upx4_imgs[index], size=224, resize=True))
    SR_img = np.array(load_SR(upx4_imgs[index]))
    real_img = np.array(load_image(real_imgs[index], resize=False))
    
    bicubic_psnr += psnr(bicubic, real_img)
    SR_psnr += psnr(SR_img, real_img)
    
    bicubic_ssim += ssim(bicubic, real_img, multichannel=True)
    SR_ssim += ssim(SR_img, real_img, multichannel=True)
    
    #print('Bicubic PSNR {0:.2f}'.format(bicubic_psnr))
    #print('SR Model PSNR {0:.2f}'.format(SR_psnr))
    #print('Bicubic SSIM {0:.2f}'.format(bicubic_ssim))
    #print('SR Model SSIM {0:.2f}'.format(SR_ssim))
    
print('Average Bicubic PSNR {0:.2f}'.format(bicubic_psnr/len(real_imgs)))
print('Average SR Model PSNR {0:.2f}'.format(SR_psnr/len(real_imgs)))
print('Average Bicubic SSIM {0:.2f}'.format(bicubic_ssim/len(real_imgs)))
print('Average SR Model SSIM {0:.2f}'.format(SR_ssim/len(real_imgs)))    

Average Bicubic PSNR 27.93
Average SR Model PSNR 27.98
Average Bicubic SSIM 0.79
Average SR Model SSIM 0.80


### SRCNN Vanilla

In [9]:
model_3_path = '../models/SRCNN_x4_epoch_50.pth'
model_3 = torch.load(model_3_path, map_location=lambda storage, loc: storage.cuda(0))
model_3 = model_3.to(device)

load_SR = SuperResolveDL(model_3, size=224, channel=1, resize=True)

In [10]:
bicubic_psnr = 0
SR_psnr = 0
bicubic_ssim = 0
SR_ssim = 0

for index in range(len(real_imgs)):
    
    bicubic = np.array(load_image(upx4_imgs[index], size=224, resize=True))
    SR_img = np.array(load_SR(upx4_imgs[index]))
    real_img = np.array(load_image(real_imgs[index], resize=False))
    
    bicubic_psnr += psnr(bicubic, real_img)
    SR_psnr += psnr(SR_img, real_img)
    
    bicubic_ssim += ssim(bicubic, real_img, multichannel=True)
    SR_ssim += ssim(SR_img, real_img, multichannel=True)
    
    #print('Bicubic PSNR {0:.2f}'.format(bicubic_psnr))
    #print('SR Model PSNR {0:.2f}'.format(SR_psnr))
    #print('Bicubic SSIM {0:.2f}'.format(bicubic_ssim))
    #print('SR Model SSIM {0:.2f}'.format(SR_ssim))
    
print('Average Bicubic PSNR {0:.2f}'.format(bicubic_psnr/len(real_imgs)))
print('Average SR Model PSNR {0:.2f}'.format(SR_psnr/len(real_imgs)))
print('Average Bicubic SSIM {0:.2f}'.format(bicubic_ssim/len(real_imgs)))
print('Average SR Model SSIM {0:.2f}'.format(SR_ssim/len(real_imgs)))    

Average Bicubic PSNR 27.93
Average SR Model PSNR 27.95
Average Bicubic SSIM 0.79
Average SR Model SSIM 0.80


### SubCNN Vanilla

In [17]:
model_4_path = '../models/SubCNN_x4_epoch_50.pth'
model_4 = torch.load(model_4_path, map_location=lambda storage, loc: storage.cuda(0))
model_4 = model_4.to(device)

load_SR = SuperResolveDL(model_4, size=224, channel=1, resize=False)

In [14]:
bicubic_psnr = 0
SR_psnr = 0
bicubic_ssim = 0
SR_ssim = 0

for index in range(len(real_imgs)):
    
    bicubic = np.array(load_image(upx4_imgs[index], size=224, resize=True))
    SR_img = np.array(load_SR(upx4_imgs[index]))
    real_img = np.array(load_image(real_imgs[index], resize=False))
    
    bicubic_psnr += psnr(bicubic, real_img)
    SR_psnr += psnr(SR_img, real_img)
    
    bicubic_ssim += ssim(bicubic, real_img, multichannel=True)
    SR_ssim += ssim(SR_img, real_img, multichannel=True)
    
    #print('Bicubic PSNR {0:.2f}'.format(bicubic_psnr))
    #print('SR Model PSNR {0:.2f}'.format(SR_psnr))
    #print('Bicubic SSIM {0:.2f}'.format(bicubic_ssim))
    #print('SR Model SSIM {0:.2f}'.format(SR_ssim))
    
print('Average Bicubic PSNR {0:.2f}'.format(bicubic_psnr/len(real_imgs)))
print('Average SR Model PSNR {0:.2f}'.format(SR_psnr/len(real_imgs)))
print('Average Bicubic SSIM {0:.2f}'.format(bicubic_ssim/len(real_imgs)))
print('Average SR Model SSIM {0:.2f}'.format(SR_ssim/len(real_imgs)))    

Average Bicubic PSNR 27.93
Average SR Model PSNR 28.08
Average Bicubic SSIM 0.79
Average SR Model SSIM 0.80


### SRCNN Face Loss

In [18]:
model_5_path = '../models/SRCNN_Loss_x4_epoch_20.pth'
model_5 = torch.load(model_5_path, map_location=lambda storage, loc: storage.cuda(0))
model_5 = model_5.to(device)

load_SR = SuperResolveDL(model_5, size=224, channel=3, resize=True)

In [19]:
bicubic_psnr = 0
SR_psnr = 0
bicubic_ssim = 0
SR_ssim = 0

for index in range(len(real_imgs)):
    
    bicubic = np.array(load_image(upx4_imgs[index], size=224, resize=True))
    SR_img = np.array(load_SR(upx4_imgs[index]))
    real_img = np.array(load_image(real_imgs[index], resize=False))
    
    bicubic_psnr += psnr(bicubic, real_img)
    SR_psnr += psnr(SR_img, real_img)
    
    bicubic_ssim += ssim(bicubic, real_img, multichannel=True)
    SR_ssim += ssim(SR_img, real_img, multichannel=True)
    
    #print('Bicubic PSNR {0:.2f}'.format(bicubic_psnr))
    #print('SR Model PSNR {0:.2f}'.format(SR_psnr))
    #print('Bicubic SSIM {0:.2f}'.format(bicubic_ssim))
    #print('SR Model SSIM {0:.2f}'.format(SR_ssim))
    
print('Average Bicubic PSNR {0:.2f}'.format(bicubic_psnr/len(real_imgs)))
print('Average SR Model PSNR {0:.2f}'.format(SR_psnr/len(real_imgs)))
print('Average Bicubic SSIM {0:.2f}'.format(bicubic_ssim/len(real_imgs)))
print('Average SR Model SSIM {0:.2f}'.format(SR_ssim/len(real_imgs)))    

Average Bicubic PSNR 27.93
Average SR Model PSNR 24.75
Average Bicubic SSIM 0.79
Average SR Model SSIM 0.77


## Task 2 - Upsample x2

### SubCNN Coord

In [17]:
# SubCNN 
model_1_path = '../models/SubCNN_coord_x2_epoch_50.pth'
model_1 = torch.load(model_1_path, map_location=lambda storage, loc: storage.cuda(0))
model_1 = model_1.to(device)

load_SR = SuperResolveDL(model_1, size=224, channel=1, resize=False)

In [18]:
bicubic_psnr = 0
SR_psnr = 0
bicubic_ssim = 0
SR_ssim = 0

for index in range(len(real_imgs)):
    
    bicubic = np.array(load_image(upx2_imgs[index], size=224, resize=True))
    SR_img = np.array(load_SR(upx2_imgs[index]))
    real_img = np.array(load_image(real_imgs[index], resize=False))
    
    bicubic_psnr += psnr(bicubic, real_img)
    SR_psnr += psnr(SR_img, real_img)
    
    bicubic_ssim += ssim(bicubic, real_img, multichannel=True)
    SR_ssim += ssim(SR_img, real_img, multichannel=True)
    
    #print('Bicubic PSNR {0:.2f}'.format(bicubic_psnr))
    #print('SR Model PSNR {0:.2f}'.format(SR_psnr))
    #print('Bicubic SSIM {0:.2f}'.format(bicubic_ssim))
    #print('SR Model SSIM {0:.2f}'.format(SR_ssim))
    
print('Average Bicubic PSNR {0:.2f}'.format(bicubic_psnr/len(real_imgs)))
print('Average SR Model PSNR {0:.2f}'.format(SR_psnr/len(real_imgs)))
print('Average Bicubic SSIM {0:.2f}'.format(bicubic_ssim/len(real_imgs)))
print('Average SR Model SSIM {0:.2f}'.format(SR_ssim/len(real_imgs)))    

Average Bicubic PSNR 32.44
Average SR Model PSNR 32.03
Average Bicubic SSIM 0.89
Average SR Model SSIM 0.89


## SRCNN Coord

In [19]:
model_2_path = '../models/SRCNN_coord_x2_epoch_50.pth'
model_2 = torch.load(model_2_path, map_location=lambda storage, loc: storage.cuda(0))
model_2 = model_2.to(device)

load_SR = SuperResolveDL(model_2, size=224, channel=1, resize=True)

In [20]:
bicubic_psnr = 0
SR_psnr = 0
bicubic_ssim = 0
SR_ssim = 0

for index in range(len(real_imgs)):
    
    bicubic = np.array(load_image(upx2_imgs[index], size=224, resize=True))
    SR_img = np.array(load_SR(upx2_imgs[index]))
    real_img = np.array(load_image(real_imgs[index], resize=False))
    
    bicubic_psnr += psnr(bicubic, real_img)
    SR_psnr += psnr(SR_img, real_img)
    
    bicubic_ssim += ssim(bicubic, real_img, multichannel=True)
    SR_ssim += ssim(SR_img, real_img, multichannel=True)
    
    #print('Bicubic PSNR {0:.2f}'.format(bicubic_psnr))
    #print('SR Model PSNR {0:.2f}'.format(SR_psnr))
    #print('Bicubic SSIM {0:.2f}'.format(bicubic_ssim))
    #print('SR Model SSIM {0:.2f}'.format(SR_ssim))
    
print('Average Bicubic PSNR {0:.2f}'.format(bicubic_psnr/len(real_imgs)))
print('Average SR Model PSNR {0:.2f}'.format(SR_psnr/len(real_imgs)))
print('Average Bicubic SSIM {0:.2f}'.format(bicubic_ssim/len(real_imgs)))
print('Average SR Model SSIM {0:.2f}'.format(SR_ssim/len(real_imgs)))    

Average Bicubic PSNR 32.44
Average SR Model PSNR 31.90
Average Bicubic SSIM 0.89
Average SR Model SSIM 0.89


### SRCNN Vanilla

In [23]:
model_3_path = '../models/SRCNN_x2_epoch_50.pth'
model_3 = torch.load(model_3_path, map_location=lambda storage, loc: storage.cuda(0))
model_3 = model_3.to(device)

load_SR = SuperResolveDL(model_3, size=224, channel=1, resize=True)

In [24]:
bicubic_psnr = 0
SR_psnr = 0
bicubic_ssim = 0
SR_ssim = 0

for index in range(len(real_imgs)):
    
    bicubic = np.array(load_image(upx2_imgs[index], size=224, resize=True))
    SR_img = np.array(load_SR(upx2_imgs[index]))
    real_img = np.array(load_image(real_imgs[index], resize=False))
    
    bicubic_psnr += psnr(bicubic, real_img)
    SR_psnr += psnr(SR_img, real_img)
    
    bicubic_ssim += ssim(bicubic, real_img, multichannel=True)
    SR_ssim += ssim(SR_img, real_img, multichannel=True)
    
    #print('Bicubic PSNR {0:.2f}'.format(bicubic_psnr))
    #print('SR Model PSNR {0:.2f}'.format(SR_psnr))
    #print('Bicubic SSIM {0:.2f}'.format(bicubic_ssim))
    #print('SR Model SSIM {0:.2f}'.format(SR_ssim))
    
print('Average Bicubic PSNR {0:.2f}'.format(bicubic_psnr/len(real_imgs)))
print('Average SR Model PSNR {0:.2f}'.format(SR_psnr/len(real_imgs)))
print('Average Bicubic SSIM {0:.2f}'.format(bicubic_ssim/len(real_imgs)))
print('Average SR Model SSIM {0:.2f}'.format(SR_ssim/len(real_imgs)))    

Average Bicubic PSNR 32.44
Average SR Model PSNR 31.90
Average Bicubic SSIM 0.89
Average SR Model SSIM 0.89


### SubCNN Vanilla

In [25]:
model_4_path = '../models/SubCNN_x2_epoch_50.pth'
model_4 = torch.load(model_4_path, map_location=lambda storage, loc: storage.cuda(0))
model_4 = model_4.to(device)

load_SR = SuperResolveDL(model_4, size=224, channel=1, resize=False)

In [26]:
bicubic_psnr = 0
SR_psnr = 0
bicubic_ssim = 0
SR_ssim = 0

for index in range(len(real_imgs)):
    
    bicubic = np.array(load_image(upx2_imgs[index], size=224, resize=True))
    SR_img = np.array(load_SR(upx2_imgs[index]))
    real_img = np.array(load_image(real_imgs[index], resize=False))
    
    bicubic_psnr += psnr(bicubic, real_img)
    SR_psnr += psnr(SR_img, real_img)
    
    bicubic_ssim += ssim(bicubic, real_img, multichannel=True)
    SR_ssim += ssim(SR_img, real_img, multichannel=True)
    
    #print('Bicubic PSNR {0:.2f}'.format(bicubic_psnr))
    #print('SR Model PSNR {0:.2f}'.format(SR_psnr))
    #print('Bicubic SSIM {0:.2f}'.format(bicubic_ssim))
    #print('SR Model SSIM {0:.2f}'.format(SR_ssim))
    
print('Average Bicubic PSNR {0:.2f}'.format(bicubic_psnr/len(real_imgs)))
print('Average SR Model PSNR {0:.2f}'.format(SR_psnr/len(real_imgs)))
print('Average Bicubic SSIM {0:.2f}'.format(bicubic_ssim/len(real_imgs)))
print('Average SR Model SSIM {0:.2f}'.format(SR_ssim/len(real_imgs)))    

Average Bicubic PSNR 32.44
Average SR Model PSNR 32.04
Average Bicubic SSIM 0.89
Average SR Model SSIM 0.88


### ProGAN Test

In [8]:
from ProGAN_model import Generator

model_path = '/home/angelo/Desktop/Github/Pytorch_Face_SR/models/generator_checkpoint_singleGPU.ckpt'

model_checkpoint = torch.load(model_path, map_location = lambda storage, loc: storage.cuda(0))
    
alpha = model_checkpoint['alpha']
step = model_checkpoint['step']

generator = Generator().to(device)
generator.load_state_dict(model_checkpoint['model_state_dict'], strict=False)

<All keys matched successfully>

In [22]:
def load_SR(path=''):
    img = Image.open(path).resize((28,28), Image.BICUBIC) 
    img = (transforms.ToTensor()(img).unsqueeze(0)).to(device)

    upsampled = ((generator(img, step, alpha)*0.5) + 0.5).squeeze(0)
    out = upsampled.cpu()
    out_img = out.data.numpy()
    out_img *= 255.0
    out_img = out_img.clip(0, 255)
    out_img = out_img.transpose((1,2,0))
    out_img = Image.fromarray(np.uint8(out_img), mode='RGB')
    
    return out_img

In [25]:
bicubic_psnr = 0
SR_psnr = 0
bicubic_ssim = 0
SR_ssim = 0

for index in range(len(real_imgs)):
    
    bicubic = np.array(load_image(upx2_imgs[index], size=224, resize=True))
    SR_img = np.array(load_SR(upx2_imgs[index]))
    real_img = np.array(load_image(real_imgs[index], resize=False))
    
    bicubic_psnr += psnr(bicubic, real_img)
    SR_psnr += psnr(SR_img, real_img)
    
    bicubic_ssim += ssim(bicubic, real_img, multichannel=True)
    SR_ssim += ssim(SR_img, real_img, multichannel=True)
    
    #print('Bicubic PSNR {0:.2f}'.format(bicubic_psnr))
    #print('SR Model PSNR {0:.2f}'.format(SR_psnr))
    #print('Bicubic SSIM {0:.2f}'.format(bicubic_ssim))
    #print('SR Model SSIM {0:.2f}'.format(SR_ssim))
    

print('Average Bicubic PSNR {0:.2f}'.format(bicubic_psnr/len(real_imgs)))
print('Average SR Model PSNR {0:.2f}'.format(SR_psnr/len(real_imgs)))
print('Average Bicubic SSIM {0:.2f}'.format(bicubic_ssim/len(real_imgs)))
print('Average SR Model SSIM {0:.2f}'.format(SR_ssim/len(real_imgs)))    

Average Bicubic PSNR 32.44
Average SR Model PSNR 17.03
Average Bicubic SSIM 0.89
Average SR Model SSIM 0.56
